# Imports

## Libraries

In [525]:
import geopandas as gpd
import pandas as pd
import geoplot as gplt
import geoplot.crs as gcrs
import mapclassify as mc
import seaborn as sns
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from benfordslaw import benfordslaw
from shapely.geometry import Point
import matplotlib.pyplot as plt

## Data

In [215]:
prov_gdf = gpd.read_file("../data/it_provinces_shapes.geojson", driver='geojson')
provinces_df = pd.read_csv("../data/provinces.csv")

# Fixing dataframes

In [216]:
# converting the "np.nan" to "NA", for naples 
provinces_df.drop(index = 107, inplace=True)
provinces_df.Code[57] = 'NA'

In [217]:
# all codes are 2 letters 
code_list = list(provinces_df.Code.unique())
for idx, x in enumerate(code_list):
#     if type(x) != str:
#         print(idx)
    if len(x) > 2:
        print(x)

In [218]:
# preparing to combine prov_gdf and provinces_df
prov_gdf['Code'] = [x[-2:] for x in prov_gdf['iso_3166_2']]

### Dropping more columns from prov_gdf

In [219]:
useless_cols = [x for x in prov_gdf.columns if (prov_gdf[x].nunique() == 1) ]
prov_gdf.drop(columns= useless_cols, inplace=True)

### Dropping columns from provinces_df

In [220]:
# useless_cols2 = [x for x in provinces_df.columns if (provinces_df[x].nunique() == 1) ]
provinces_df.drop(columns = ['Capital', 'Comuni', 'President'], inplace=True)

## Sardinian regions

### Changing prov_gdf

In [221]:
sard_geo = prov_gdf[64:72]
# sard_geo.plot()

In [248]:
sard_geo

,adm1_code,iso_3166_2,name,gns_region,geometry,Code
64,ITA-5371,IT-CI,Carbonia-Iglesias,None,"MULTIPOLYGON (((8.45590 39.07355, 8.45802 39.0...",South_Sardinia
65,ITA-5374,IT-SS,Sassari,IT14,"MULTIPOLYGON (((8.27475 41.05728, 8.25115 41.0...",Sass
66,ITA-5376,IT-NU,Nuoro,IT14,"POLYGON ((9.74884 40.66034, 9.75318 40.60529, ...",NU
67,ITA-5375,IT-OT,Olbia-Tempio,None,"MULTIPOLYGON (((8.81125 40.94083, 8.82252 40.9...",Sass
68,ITA-5373,IT-OR,Oristrano,IT14,"POLYGON ((8.54852 39.69791, 8.56577 39.70108, ...",OR
69,ITA-5372,IT-VS,Medio Campidano,IT14,"POLYGON ((8.39806 39.44650, 8.38950 39.46235, ...",South_Sardinia
70,ITA-5378,IT-CA,Cagliari,None,"POLYGON ((9.64630 39.54909, 9.62843 39.50178, ...",South_Sardinia
71,ITA-5377,IT-OG,Ogliastra,None,"POLYGON ((9.62718 40.22452, 9.62908 40.20555, ...",South_Sardinia


In [222]:
prov_gdf['Code'][64] = 'South_Sardinia'
prov_gdf['Code'][65] = 'Sass'
prov_gdf['Code'][67] = 'Sass'
prov_gdf['Code'][69] = 'South_Sardinia'
prov_gdf['Code'][70] = 'South_Sardinia'
prov_gdf['Code'][71] = 'South_Sardinia'

In [257]:
# merging the geometries of the new "regions"
prov_gdf2 = prov_gdf.dissolve(by='Code')

### Changing Provinces_df

In [249]:
provinces_df.columns

Index(['Type', 'Province', 'Code', 'Region', 'Macroregion',
       'Population (2019)[3]', 'Area(km²)', 'Density(p/km²)'],
      dtype='object')

In [224]:
# provinces_df[provinces_df.Region == 'Sardinia']
## new population for south_sardinia is 431038 + 350725 = 781763
provinces_df['Code'][17] = 'South_Sardinia'
provinces_df['Code'][83] = 'Sass'
provinces_df.drop(index = 87, inplace=True)
# provinces_df['Code'][87] = 'South_Sardinia'
provinces_df['Population (2019)[3]'][17] = '781,763'
# provinces_df['Population (2019)[3]'][87] = '781,763'

## Merging the 2 Dataframes

In [225]:
provinces_df.shape

(106, 8)

In [250]:
prov_gdf.head(2)

,adm1_code,iso_3166_2,name,gns_region,geometry,Code
0,ITA-5442,IT-AO,Aoste,IT19,"POLYGON ((7.02208 45.92526, 7.06694 45.89022, ...",AO
1,ITA-5437,IT-VB,Verbano-Cusio-Ossola,IT12,"POLYGON ((7.84962 45.93971, 7.87020 45.94037, ...",VB


In [258]:
provinces_data_gdf = prov_gdf2.merge(provinces_df, on='Code', how='inner')

In [259]:
provinces_data_gdf['Population_no_commas'] = [x.replace(',', '') 
                                              for x in provinces_data_gdf['Population (2019)[3]']]
provinces_data_gdf['Population'] = provinces_data_gdf['Population_no_commas'].astype(float)

In [ ]:
# Imports

import geopandas as gpd
import pandas as pd
import geoplot as gplt
import geoplot.crs as gcrs
import mapclassify as mc
import seaborn as sns
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from benfordslaw import benfordslaw
from shapely.geometry import Point
import matplotlib.pyplot as plt

prov_gdf = gpd.read_file("../data/it_provinces_shapes.geojson", driver='geojson')
provinces_df = pd.read_csv("../data/provinces.csv")

pip install benfordslaw

# Fixing dataframes

# fixing provinces_df
provinces_df.drop(index = 107, inplace=True)
provinces_df.Code[57] = 'NA'

# all codes are 2 letters 
code_list = list(provinces_df.Code.unique())
for idx, x in enumerate(code_list):
#     if type(x) != str:
#         print(idx)
    if len(x) > 2:
        print(x)

# preparing to combine prov_gdf and provinces_df
prov_gdf['Code'] = [x[-2:] for x in prov_gdf['iso_3166_2']]

### Dropping more columns from prov_gdf

useless_cols = [x for x in prov_gdf.columns if (prov_gdf[x].nunique() == 1) ]
prov_gdf.drop(columns= useless_cols, inplace=True)

### Dropping columns from provinces_df

# useless_cols2 = [x for x in provinces_df.columns if (provinces_df[x].nunique() == 1) ]
provinces_df.drop(columns = ['Capital', 'Comuni', 'President'], inplace=True)

## Sardinian regions

### Changing prov_gdf

sard_geo = prov_gdf[64:72]
# sard_geo.plot()

sard_geo

prov_gdf['Code'][64] = 'South_Sardinia'
prov_gdf['Code'][65] = 'Sass'
prov_gdf['Code'][67] = 'Sass'
prov_gdf['Code'][69] = 'South_Sardinia'
prov_gdf['Code'][70] = 'South_Sardinia'
prov_gdf['Code'][71] = 'South_Sardinia'

# merging the geometries of the new "regions"
prov_gdf2 = prov_gdf.dissolve(by='Code')

### Changing Provinces_df

provinces_df.columns

# provinces_df[provinces_df.Region == 'Sardinia']
## new population for south_sardinia is 431038 + 350725 = 781763
provinces_df['Code'][17] = 'South_Sardinia'
provinces_df['Code'][83] = 'Sass'
provinces_df.drop(index = 87, inplace=True)
# provinces_df['Code'][87] = 'South_Sardinia'
provinces_df['Population (2019)[3]'][17] = '781,763'
# provinces_df['Population (2019)[3]'][87] = '781,763'

## Merging the 2 Dataframes

provinces_df.shape

prov_gdf.head(2)

provinces_data_gdf = prov_gdf2.merge(provinces_df, on='Code', how='inner')

provinces_data_gdf['Population_no_commas'] = [x.replace(',', '') 
                                              for x in provinces_data_gdf['Population (2019)[3]']]
provinces_data_gdf['Population'] = provinces_data_gdf['Population_no_commas'].astype(float)